In [ ]:
import numpy as np
import torch
import os
import cv2
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet34
from google.colab import auth
from googleapiclient.discovery import build
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

Сегодня мы будем использовать transfer learning чтобы обучить классификатор героев классических звёздных войн. Эта практика требует обучения на GPU (иначе код будет выполнятся очень долго), поэтому используйте https://colab.research.google.com.

Сначала нужно включить поддержку GPU. Зайдите в Edit -> Notebook settings и как hardware accelerator укажите GPU.

Теперь нужно скачать данные, выполните следующую ячейку.
В первый раз она попросит вас перейти по ссылке - откройте эту ссылку и скопируйте оттуда токен (длинную строку в base64), вставьте этот токен в окошко под ячейкой.

In [ ]:
def download_data(file_id, file_name):
  import io
  from googleapiclient.http import MediaIoBaseDownload

  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()
    
  downloaded.seek(0)
  with open(file_name, "wb") as f:
    f.write(downloaded.read())

  
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_id = '139wA_Z9kustXy54ifhWWHJvARo5f7O6y'
file_name = 'star_wars.tar.gz'

download_data(file_id, file_name)
!tar xf star_wars.tar.gz

Важная часть обучения нейронных сетей (как и обычно в машинном обучении)  - это работа с данными. Необходимо представить данные в векторном виде и отдать их модели для обучения. В фреймворке Pytorch для этого используется связка двух классов: https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset и https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader. 

Первый это даже не класс, а интерфейс - у него всего два интересных метода:
__getitem__ - выбрать из датасета элемент по заданному числовому индексу
__len__ - вернуть длину датасета.



DataLoader в свою очередь занимается тем что выбирает элементы из Dataset который передается ему в конструкторе и делает из них батчи для обучения нейронной сети. Вы можете контролировать размер батча с помощью параметра batch_size, также не забудьте включать перемешивание данных перед каждой эпохой передавая в DataLoader параметр shuffle=True. В более продвинутых случаях возможно задавать стратегию того как именно конструируются батчи с помощью классов Sampler и BatchSampler, но мы в нашей практике этих возможностей касаться не будем.

Главное отличие Dataset от массива на практике в том что этот интерфейс не обязывает вас хранить весь набор данных в памяти. Типичная реализация по индексу подгружает данные с диска и делает из них сэмпл. 

Я создал 4 папки и скачал в них картинки с помощью https://github.com/hardikvasa/google-images-download

Это делается очень просто:

```
pip install google_images_download
googleimagesdownload -k "люк скайуокер"
```
Гугл ограничивает количество картинок, которые можно скачать по поисковому запросу, но это можно обойти либо скачивать с других мест, где нет ограничений (напр. Yandex или Instagram)


В итоге у нас получилась следующая структура на диске:

```
root/люк скайуокер/xxx.jpg
root/люк скайуокер/xxy.jpeg
root/люк скайуокер/xxz.png

root/чубакка/123.png
root/чубакка/nsdf3.jpg
root/чубакка/asd932_.gif
```

По сколько это просто выдача гугла, тут встречаются как .jpg и .png так и .gif

Удобнее всего (по крайней мере мне), когда у вас есть 2 массива - один с путями к файлам и другой с лейблами для них. Давайте я сделаю его за вас:

In [ ]:
filenames = []
labels = []
for idx, class_dir in enumerate(os.listdir("star_wars")):
  print(f"берем файлы из папки \"{class_dir}\" и даем им класс {idx}")
  
  # не берем файлы кроме .jpg .jpeg и .png
  for file in os.listdir(os.path.join("star_wars", class_dir)):
    if not file.endswith(('.jpg', '.jpeg', '.png')):
      continue
      
    filenames.append(os.path.join("star_wars", class_dir, file))
    labels.append(idx)

Это типичная ситуация, как у вас могут оказаться данные на руках (а не непонятный массив как было c MNIST)

In [ ]:
# будем брать по 1 рандомной картинке, выводить ее и смотреть на класс. Так в реально жизни можно проверить не *** ли вы скачали и верные ли классы
# можете выполнять этот cell пока не надоест
random_index = np.random.choice(range(len(filenames)))
test_img = cv2.imread(filenames[random_index])[:, :, ::-1] # эта штука в конце равносильно переводу из BGR в RGB
print(labels[random_index])
plt.imshow(test_img)

In [ ]:
# Задание 1. Разбейте filenames и labels на train и test части 70/30
train_filenames, test_filenames, train_labels, test_labels = 

Теперь нашей основной задачей является создание Dataset и Dataloader для наших данных. Их должно быть два: один для train, другой для test.

Мы собираемся использовать transfer learning - взять сеть предобученную на ImageNet и доучить её на наших изображениях. Сети тренировавшиеся на ImageNet требуют стандартного размера картинок: 224x224.

In [ ]:
# Задание 2. Реализуйте класс-наследник Dataset. Он должен возвращать по индексу 
class StarWarsDataset(Dataset):
    def __init__(self, filenames, labels):
        # something here
        # something here

    def __len__(self):
        return # something here

    def __getitem__(self, idx):
        # dataloader запросит какой-то индекс (но не больше чем значение __len__)
        # мы должны отдать ему image в виде массива и соотвуствующий ему label
        filename = self._filenames[idx]
        label = # something here
        
        # мы получили имя файла, теперь нужно загрузить картинку как numpy array 
        # и изменить размер так, чтобы он был 224 на 224
        # from torchvision.io import read_image, ImageReadMode
        # preprocess_image = ResNet34_Weights.DEFAULT.transforms()
        # ...

Мы написали типичный класс-наследник Dataset для данных такого формата и поресайзили изображение к размеру 224х224 + преобразовали его в тензор внутри него. Теперь нужно создать Dataloader - да штука, которая просит возвращать Dataset данные по idx и составляет их них батчи.

In [ ]:
train_dataset = StarWarsDataset(train_fns, train_lbls)
train_dataloder = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=0)

In [ ]:
# Задание 3. Сделайте dataloader для test
test_dataset = # something here
test_dataloder = # something here

Завайте посмотрим что выводит ваш тестовый Dataloader...

In [ ]:
for batch in test_dataloder: # получаем 1 batch - 1 итерация подгрузки данных
  images, labels = batch     # наш Dataset возвращает tuple, поэтому мы можем сделать так
  print(f'Всего батчей по batch_size: {len(train_dataloder)}')
  print(f'Лейбл первого элемента в первом батче: {labels[0]}')
  print(f'Размер картинки в первом батче: {images[0].shape}')
  print(f'Картинка в первом батче: {images[0]}')
  break

In [ ]:
# Здесь мы берем предобученный resnet34 и заменяем в нём последний слой на голову классифицирующую изображение на 1 из 4 классов
# Берём кросс энтропию в качестве лосса и оптимизатор адам

model = resnet34(pretrained=True) # resnet обученный на ImageNet

# loss и optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.004)

In [ ]:
# можно принтануть модель и уведеть какие в ней слои
print(model)

Как я и говорил, transfer learning - это очень просто. В resnet куча сверток и пулингов, в конце fully connected слой на 1000. Мы удаляет последний слой, заменяя его новым на с выходом на 4 класса. Так, свертки внутри модели помнят все фичи умеют распознавать классы из ImageNet, мы просто помогаем их скорректировать на наших данных  

In [ ]:
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 4)
model.to('cuda')

In [ ]:
#Эта функция считает точность модели - на вход передается сама модель, номер эпохи и тестовый лоадер.

def run_test_on_epoch(model, epoch, test_loader):
    model.eval()
    with torch.no_grad():
      predicted_labels = []
      correct_labels = []
      for batch_x, batch_y in tqdm(test_loader):
          outputs = model(batch_x.to('cuda')).detach().cpu().numpy()
          predicted_labels.extend(np.argmax(outputs, axis=1))
          correct_labels.append(batch_y.detach().cpu().numpy())
      print("Epoch", epoch, "test accuracy", accuracy_score(np.hstack(correct_labels), np.asarray(predicted_labels)))
    model.train()

In [ ]:
# Задание 5. Напишите код для обучения модели 25 эпох. В конце каждой эпохи вызывайте run_test_on_epoch() чтобы следить за точностью
for epoch in tqdm(range(25)):
  # что-то очень важное здесь
  # строчек 6 примерно
  
  
  # половину сделал за вас
  run_test_on_epoch(model, epoch, test_dataloder)
       

Какая точность у вас получилась?

Дополнительные задания:
 1. Скачать свои данные и обучить на них
 2. Добавить агументации в класс StarWarsDataset (например, не только resize и add_pad а конвертацию в grayscale в 10% случаях или сглаживание)